# The Notebook is laid out as follows:
### 1) Loading and cleaning the Dataset
### 2) Pre Processing Data
### 3) Tokenization
### 4) Building the LSTM Model
### 5) Generating fake text from the model
### 6) Conclusion

##### 7) Trying out GPT2-HugginFace(Still in work)

### Installing dependencies

In [1]:
import os
import tensorflow as tf
import requests
import string

import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Dense,Flatten,LSTM,Embedding,Dropout,Bidirectional


## Loading Dataset

In [42]:
response = requests.get('https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt')
#response.text

Removing all (\n)'s

In [3]:
text = response.text.split('\n')
print("Number of lines {}".format(len(text)))

Number of lines 167205


Creating a single variable with all lines


In [4]:
text = ' '.join(text)
print(text[:250])

First Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.  First Citizen: You are all resolved rather to die than to famish?  All: Resolved. resolved.  First Citizen: First, you know Caius Marcius is chief enemy to the people. 


## Pre-Processing data

In [5]:
def text_processing(text):
  
    """ Pre Processing the data 

    Args: 
      text: Sequence of data to be pre-processed  

    Returns:
      tokens: Returns cleaned data that is aplhanumeric and in lower case.
    """


  tokens = text.split()
  table = str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]

  return tokens


In [6]:
tokens = text_processing(text)
tokens[:10]


['first',
 'citizen',
 'before',
 'we',
 'proceed',
 'any',
 'further',
 'hear',
 'me',
 'speak']

In [7]:
prev_seq = 50 + 1 #Number of previous words the model will consider to predict the next word
sentances = []
for i in range(0,len(tokens)):
  seq = tokens[i:i+prev_seq] #Creating sequences of 50 words each
  sentance = ' '.join(seq) # Joining tokens of same sequence
  sentances.append(sentance) #Appending all sentances to one list

  if i > 200000:
    break
print(len(sentances))
sentances[0]

200002


'first citizen before we proceed any further hear me speak all speak speak first citizen you are all resolved rather to die than to famish all resolved resolved first citizen first you know caius marcius is chief enemy to the people all we knowt we knowt first citizen let us kill'

## Tokenization

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentances) # Every unique char will be assigned an integer 
sequence = np.array(tokenizer.texts_to_sequences(sentances)) # Replaces characters with their tokenizer output from above function

In [9]:
vocab = tokenizer.word_index #Gives all Word:Index mapping
vocab_length = len(vocab) + 1
print(vocab_length)


12750


In [10]:
X = sequence[:,:-1]  # All columns except the last (50 prev words to predict 51st(Seq length was 51))
y = sequence[:,-1]  #Select the last columns only(Word to be predicted)

In [11]:
y = to_categorical(y,num_classes=vocab_length)
seq_length = X.shape[1]
print(seq_length)

50


## Building RNN

In [12]:
def create_model(vocab_length,seq_length):
    
    """ Creating the LSTM based RNN

    Args: 
      vocab_length: Lenght of the vocabulary(Unique word : index mapping)
      seq_length: Length of each sample

    Returns:
      model: Model
    """

    model = Sequential()
    model.add(Embedding(vocab_length,50,input_length=seq_length))
    model.add((LSTM(100,return_sequences = True)))
    model.add((LSTM(100)))
    #model.add(Dropout(0.1))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(vocab_length,activation='softmax'))

    return model


In [13]:
model = create_model(vocab_length,seq_length)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            637500    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 12750)             1287750   
Total params: 2,076,150
Trainable params: 2,076,150
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])


In [15]:
model.fit(X,y,batch_size = 256,epochs = 100)

Epoch 1/100
782/782 [==============================] - 25s 32ms/step - loss: 6.8737 - accuracy: 0.0312
Epoch 2/100
782/782 [==============================] - 24s 31ms/step - loss: 6.5192 - accuracy: 0.0406
Epoch 3/100
782/782 [==============================] - 24s 31ms/step - loss: 6.3209 - accuracy: 0.0573
Epoch 4/100
782/782 [==============================] - 24s 30ms/step - loss: 6.1362 - accuracy: 0.0720
Epoch 5/100
782/782 [==============================] - 25s 31ms/step - loss: 6.0026 - accuracy: 0.0813
Epoch 6/100
782/782 [==============================] - 25s 31ms/step - loss: 5.8913 - accuracy: 0.0868
Epoch 7/100
782/782 [==============================] - 24s 31ms/step - loss: 5.7965 - accuracy: 0.0911
Epoch 8/100
782/782 [==============================] - 25s 32ms/step - loss: 5.7127 - accuracy: 0.0945
Epoch 9/100
782/782 [==============================] - 24s 31ms/step - loss: 5.6338 - accuracy: 0.0966
Epoch 10/100
782/782 [==============================] - 24s 30ms/step - l

782/782 [==============================] - 23s 29ms/step - loss: 3.4994 - accuracy: 0.2887
Epoch 80/100
782/782 [==============================] - 23s 30ms/step - loss: 3.4858 - accuracy: 0.2899
Epoch 81/100
782/782 [==============================] - 23s 29ms/step - loss: 3.4716 - accuracy: 0.2926
Epoch 82/100
782/782 [==============================] - 24s 30ms/step - loss: 3.4576 - accuracy: 0.2942
Epoch 83/100
782/782 [==============================] - 23s 29ms/step - loss: 3.4429 - accuracy: 0.2968
Epoch 84/100
782/782 [==============================] - 23s 29ms/step - loss: 3.4340 - accuracy: 0.2976
Epoch 85/100
782/782 [==============================] - 23s 29ms/step - loss: 3.4166 - accuracy: 0.2993
Epoch 86/100
782/782 [==============================] - 23s 29ms/step - loss: 3.4026 - accuracy: 0.3024
Epoch 87/100
782/782 [==============================] - 23s 30ms/step - loss: 3.3911 - accuracy: 0.3043
Epoch 88/100
782/782 [==============================] - 23s 29ms/step - loss:

## Generating Fake Text

In [16]:
def generate_text(model,tokenizer,text_seq_length,seed_text,n_words):
    
    
    """ Generating Fake text using the model trained

    Args:
      model: RNN trained 
      tokenizer: Tokenizer object for preprocessing
      text_seq_length: Length of each sample
      seed_text: starting text based on which model will generate new samples
      n_words: Number of sample to generate

    Returns:
      model: Returns sequence of 100 predicted words
    """


    pred_text = []
    
    for _ in range(n_words):
        
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded],maxlen = 50,truncating = 'pre')
        
        y_pred = model.predict_classes(encoded)
        print(y_pred)
        #print(y_pred)
        
        word_y = ''
        for word,index in tokenizer.word_index.items():
            #print(index,y_pred)
            if index == y_pred:
                word_y = word
                #print(word_y)
                break
            
        seed_text = seed_text + ' ' + word_y
        pred_text.append(word_y)
        #print(pred_text)
    return ' '.join(pred_text)
            
        

### Generating new text after the first 50 words

In [17]:
generate_text(model,tokenizer,seq_length,sentances[0],100)

[26]
[4760]
[33]
[257]
[7]
[44]
[4]
[68]
[8]
[234]
[5]
[107]
[2]
[208]
[20]
[4]
[68]
[10522]
[6420]
[13]
[6]
[11]
[9078]
[84]
[12]
[17]
[140]
[714]
[20]
[6]
[36]
[202]
[1]
[3154]
[5]
[1]
[1948]
[1]
[569]
[4]
[23]
[8]
[234]
[540]
[395]
[4]
[30]
[12]
[17]
[793]
[3305]
[273]
[4]
[23]
[147]
[8]
[2135]
[963]
[3]
[17]
[8]
[12518]
[2]
[11886]
[18]
[1482]
[2096]
[67]
[123]
[6]
[23]
[6529]
[6]
[41]
[12]
[538]
[6]
[2]
[4]
[68]
[8]
[234]
[540]
[3]
[17]
[8]
[4861]
[5]
[18]
[190]
[2]
[81]
[24]
[1651]
[24]
[1]
[3879]
[90]
[358]
[60]


'him mariner no madam my lord i am a gentleman of mine and nothing but i am galledmightst savours for you is weandi must not be thus bold but you shall bear the burden of the sky the heavens i have a gentleman born clown i will not be consul sixth citizen i have been a stranger glad to be a rabbit and paind your unknown sovereignty duke vincentio you have confessed you do not mark you and i am a gentleman born to be a surplus of your grace and yet as glorious as the fliers first murderer how'

### Actual text following the first 50 words

In [18]:
sentances[1]

'citizen before we proceed any further hear me speak all speak speak first citizen you are all resolved rather to die than to famish all resolved resolved first citizen first you know caius marcius is chief enemy to the people all we knowt we knowt first citizen let us kill him'

## Conclusion
### The output generated by LSTM's are pretty coherent to some extent. 

### The drawback of LSTM is that, at each time step we are predicting only one entity. Secondly, the performance of LSTM's worsens as we try to predict larger sequences(Vanishing Gradient).

### Alternatives to LSTM can includes 2D convolutional based neural network with causal convolution and attention based encoder-decoder models which can work for much longer sequences.

* ---------------------------------------------------------------------------------------------------------------------------------------------- *

## Trying out GPT2-HuggingFace

In [20]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [21]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)


GPT2.summary()



Model: "tfgp_t2lm_head_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFGPT2MainLayer multiple                  774030080 
Total params: 774,030,080
Trainable params: 774,030,080
Non-trainable params: 0
_________________________________________________________________


In [40]:
def generate_GPT(model,tokenizer,seed_text,n_words):
    
    
    encoded = tokenizer.encode(seed_text, return_tensors='tf')
    
    sample_outputs = model.generate(
                              encoded,
                              do_sample = True, 
                              max_length = n_words,
                              temperature = 0.7,
                              top_k = 50, 
                              top_p = 0.85 
                              )

    print("Output:\n" + 100 * '-')
    for i, sample_output in enumerate(sample_outputs):
        print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
        print('')

In [41]:
generate_GPT(GPT2,tokenizer,'If Antonio fails to pay Shylock back, Shylock demands a pound of Antonios flesh.',100)

Output:
----------------------------------------------------------------------------------------------------
0: If Antonio fails to pay Shylock back, Shylock demands a pound of Antonios flesh.

Antonios refuses and Shylock uses his powers to force him to eat the flesh. Shylock then forces the man to confess to the murder of his wife and children. The man is forced to admit that he killed the children because he was angry that the woman he loved was cheating on him. The man then admits to the murder of his wife and children and begs for...

